In [ ]:
#currently this notebook is WIP and not working fully
#will be sorting out the graphframe import in another update.

#TODO:fix graphframes import

In [ ]:

import pyspark
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [ ]:
import networkx as nx

In [ ]:
#util functions to find where splink_graph is installed and create a
#suitable string to put on spark.jars configuration so graphframes work.

from splink_graph.utils import _get_local_site_packages_dir, _create_spark_jars_string



In [ ]:
localsite=_get_local_site_packages_dir()
localsite

In [ ]:
sjstring = _create_spark_jars_string().replace(" ", "")
sjstring

In [ ]:

    

spark = (
    SparkSession.builder.appName('GraphUtils').master('local[*]')
    #change with your spark intallationconfigs here.These are for MOJ AP
    
    .config("spark.jars.ivy", "/home/jovyan/.ivy2/")
    
    
    #these configs are required for splink_graph and graphframes to run properly 
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    
    .config("spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT", "1")
    .config('spark.jars',sjstring)
    .getOrCreate()
  )

In [ ]:
from graphframes import *


In [ ]:
# Create a Vertex DataFrame with unique ID column "id"
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Dennis", 30),    
  ("e", "Ernest", 30), 
  ("f", "Ford", 30),
  ("g", "George", 30), 
  ("h", "Harry", 30),
  ("i", "Ion", 30),
  ("j", "John", 30),
    
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = spark.createDataFrame([
  ("a", "b", 0.4),
  ("b", "c", 0.56),
 
  
  ("d", "e", 0.84),
  ("e", "f", 0.65),
  ("f", "d", 0.67),
  ("f", "g", 0.34),
  ("g", "h", 0.99),
  ("h", "i", 0.5),
  ("h", "j", 0.8),]
    
, ["src", "dst", "similarity"])
# Create a GraphFrame

g = GraphFrame(v, e)

In [ ]:
# Check if graphframes library works
g.inDegrees.show()

In [ ]:
# create a function to plot a graphframe graph via networkx and matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

def PlotGraph(g):
    edge_list = g.edges

    Gplot=nx.Graph()
    for row in edge_list.select('src','dst','similarity').take(1000):
        Gplot.add_edge(row['src'],row['dst'],weight=row['similarity'])
        # labels=nx.draw_networkx_labels(Gplot,pos=nx.spring_layout(Gplot))

    plt.subplot(121)
    nx.draw(Gplot,with_labels=True,font_weight='bold')

In [ ]:
PlotGraph(g)